In [1]:
import os
import json
import pandas as pd
import requests
from ray import serve
from transformers import pipeline
from starlette.requests import Request
from collections import defaultdict
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
sentiment = pipeline(
    "sentiment-analysis",
    "distilbert-base-uncased-finetuned-sst-2-english"
)

In [3]:
sentiment("I think what is meeting was very cool")

[{'label': 'POSITIVE', 'score': 0.998663067817688}]

In [4]:
@serve.deployment
class Summarizer:
    def __init__(self):
        self.model = "distilbert-base-uncased-finetuned-sst-2-english"
        self.sentiment = pipeline("sentiment-analysis", model=self.model)

    async def __call__(self, request: Request):
        query_params = request.query_params
        headers = defaultdict(str, request.headers)
        if headers["content-type"] == "application/json" and "txt" in query_params:
            raise NameError("Invalid request Schema")
        if headers["content-type"] == "application/json":
            payload = await request.json()
            payload = json.loads(payload)
        else:
            payload = [{"text": query_params["txt"]}]
        sent_list = [p["text"] for p in payload]
        batch = self.sentiment(sent_list)
        labels = [item["label"] for item in batch]
        return labels

In [5]:
serve.start()
Summarizer.deploy()

2023-06-18 14:40:09,948	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(ServeController pid=1259) INFO 2023-06-18 14:40:11,535 controller 1259 deployment_state.py:1298 - Deploying new version of deployment Summarizer.
(HTTPProxyActor pid=1307) INFO:     Started server process [1307]
(ServeController pid=1259) INFO 2023-06-18 14:40:11,626 controller 1259 deployment_state.py:1537 - Adding 1 replica to deployment Summarizer.


In [6]:
text = "I think what is meeting was very cool"
response = requests.get(f"http://127.0.0.1:8000/Summarizer?txt={text}")
response.text

'["POSITIVE"]'

In [7]:
df = pd.DataFrame({
    "text": [
        "Very positive sentence", 
        "Very negative sentence"
    ]
})
df

,text
0,Very positive sentence
1,Very negative sentence


In [8]:
df.to_json(orient="records")

'[{"text":"Very positive sentence"},{"text":"Very negative sentence"}]'

In [10]:
json = df.to_json(orient="records")
response = requests.get(f"http://127.0.0.1:8000/Summarizer", json=json)
response.text

'["POSITIVE", "NEGATIVE"]'

(ServeReplica:Summarizer pid=1367) INFO 2023-06-18 14:40:17,097 Summarizer Summarizer#MchoWb eNrMMzGnGA /Summarizer replica.py:654 - __CALL__ OK 51.5ms
